In [2]:
## first time series test
# append ../../ to make link to main repo work
import sys
sys.path.append('../../')

# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex


import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [3]:
# import image libraries
from PIL import Image, ImageDraw, ImageFont
import os
# import cv2
import numpy as np

In [4]:
# load the data

WIDTH=18158
nlines = 7932

Image.MAX_IMAGE_PIXELS = WIDTH*nlines


CSK_RMLI_ras = glob.glob('/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/*')
TSX_RMLI_ras = glob.glob('/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/*')
CSK_RMLI_ras.sort()
TSX_RMLI_ras.sort()
TSX_RMLI_ras = TSX_RMLI_ras[1:]

CSK_dates = [datetime.datetime.strptime(file[-17:-9],'%Y%m%d') for file in CSK_RMLI_ras]
TSX_dates = [datetime.datetime.strptime(file[-16:-8],'%Y%m%d') for file in TSX_RMLI_ras]

print(CSK_dates)
print(TSX_dates)

print(np.shape(CSK_RMLI_ras))
print(TSX_RMLI_ras)
print(np.shape(TSX_RMLI_ras))

# CSK_RMLI_ras = ['/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200910.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200919.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200926.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200927.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201005.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201012.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201013.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201113.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201114.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201122.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201224.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201231.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20210101.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20210109.rmli.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20211101.rmli.ras']

# TSX_RMLI_ras = ['/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201116.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201127.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201208.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201219.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210110.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210201.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210223.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210317.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210408.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210430.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210522.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210602.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210613.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20211001.pwr.ras',
#                 '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20211023.pwr.ras']


[datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2020, 9, 19, 0, 0), datetime.datetime(2020, 9, 26, 0, 0), datetime.datetime(2020, 9, 27, 0, 0), datetime.datetime(2020, 10, 5, 0, 0), datetime.datetime(2020, 10, 12, 0, 0), datetime.datetime(2020, 10, 13, 0, 0), datetime.datetime(2020, 11, 13, 0, 0), datetime.datetime(2020, 11, 14, 0, 0), datetime.datetime(2020, 11, 22, 0, 0), datetime.datetime(2020, 12, 24, 0, 0), datetime.datetime(2020, 12, 31, 0, 0), datetime.datetime(2021, 1, 1, 0, 0), datetime.datetime(2021, 1, 9, 0, 0), datetime.datetime(2021, 1, 16, 0, 0), datetime.datetime(2021, 1, 17, 0, 0), datetime.datetime(2021, 1, 25, 0, 0), datetime.datetime(2021, 2, 1, 0, 0), datetime.datetime(2021, 2, 2, 0, 0), datetime.datetime(2021, 2, 17, 0, 0), datetime.datetime(2021, 2, 18, 0, 0), datetime.datetime(2021, 3, 2, 0, 0), datetime.datetime(2021, 3, 5, 0, 0), datetime.datetime(2021, 3, 6, 0, 0), datetime.datetime(2021, 3, 14, 0, 0), datetime.datetime(2021, 3, 21, 0, 0), datetime.da

In [5]:
def load_imag(filename):
        """ funtion to load image files using Pillow
        """
        with Image.open(filename) as image1_pil:
                image = np.asarray(image1_pil)
        return image

In [18]:
%matplotlib osx
import string
# display image
plt.close('all')

fig = plt.figure(figsize=(8,8))

gs = GridSpec(4, 4, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # 
ax01 = fig.add_subplot(gs[0, 1]) # 
ax02 = fig.add_subplot(gs[0, 2]) # 
ax03 = fig.add_subplot(gs[0, 3]) # 


ax10 = fig.add_subplot(gs[1, 0]) # 
ax11 = fig.add_subplot(gs[1, 1]) # 
ax12 = fig.add_subplot(gs[1, 2]) # 
ax13 = fig.add_subplot(gs[1, 3]) # 

ax20 = fig.add_subplot(gs[2, 0]) # 
ax21 = fig.add_subplot(gs[2, 1]) # 
ax22 = fig.add_subplot(gs[2, 2]) # 
ax23 = fig.add_subplot(gs[2, 3]) # 

ax30 = fig.add_subplot(gs[3, 0]) # 
ax31 = fig.add_subplot(gs[3, 1]) # 
ax32 = fig.add_subplot(gs[3, 2]) # 
ax33 = fig.add_subplot(gs[3, 3]) # 


axes = [ax00,ax01,ax02,ax03,ax10,ax11,ax12,ax13,ax20,ax21,ax22,ax23,ax30,ax31,ax32,ax33]
labels = string.ascii_uppercase[0:len(axes)]
label_x = [0.03,0.03,0.03,0.03,
           0.03,0.03,0.03,0.03,
           0.03,0.03,0.03,0.03,
           0.03,0.03,0.03,0.03]
label_y = [0.86,0.86,0.86,0.86,
           0.86,0.86,0.86,0.86,
           0.86,0.86,0.86,0.86,
           0.86,0.86,0.86,0.86]

bbox = dict(fc="1.0",alpha=0.5)
for (a,img,date,label,lab_x,lab_y) in zip(axes,CSK_RMLI_ras[0::3],CSK_dates[0::3],labels,label_x,label_y):
    image = load_imag(img)
    image_crop = image[4200:6000,1300:3100]
    a.imshow(np.fliplr(np.rot90(image_crop,3)),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.annotate(date.strftime('%Y-%m-%d'),xy=[lab_x+0.0,lab_y-0.795],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.set_axis_off()
ax33.set_axis_off()
fig.subplots_adjust(wspace=0.01,hspace=0.01)



In [6]:
%matplotlib osx
import string
# display image
plt.close('all')

fig = plt.figure(figsize=(8,8))

gs = GridSpec(4, 4, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # 
ax01 = fig.add_subplot(gs[0, 1]) # 
ax02 = fig.add_subplot(gs[0, 2]) # 
ax03 = fig.add_subplot(gs[0, 3]) # 


ax10 = fig.add_subplot(gs[1, 0]) # 
ax11 = fig.add_subplot(gs[1, 1]) # 
ax12 = fig.add_subplot(gs[1, 2]) # 
ax13 = fig.add_subplot(gs[1, 3]) # 

ax20 = fig.add_subplot(gs[2, 0]) # 
ax21 = fig.add_subplot(gs[2, 1]) # 
ax22 = fig.add_subplot(gs[2, 2]) # 
ax23 = fig.add_subplot(gs[2, 3]) # 

ax30 = fig.add_subplot(gs[3, 0]) # 
ax31 = fig.add_subplot(gs[3, 1]) # 
ax32 = fig.add_subplot(gs[3, 2]) # 
ax33 = fig.add_subplot(gs[3, 3]) # 


axes = [ax00,ax01,ax02,ax03,ax10,ax11,ax12,ax13,ax20,ax21,ax22,ax23,ax30,ax31,ax32,ax33]
labels = string.ascii_uppercase[0:len(axes)]
label_x = [0.02,0.02,0.02,0.02,
           0.02,0.02,0.02,0.02,
           0.02,0.02,0.02,0.02,
           0.02,0.02,0.02,0.02]
label_y = [0.87,0.87,0.87,0.87,
           0.87,0.87,0.87,0.87,
           0.87,0.87,0.87,0.87,
           0.87,0.87,0.87,0.87]

bbox = dict(fc="1.0",alpha=0.5)
for (a,img,date, label,lab_x,lab_y) in zip(axes,TSX_RMLI_ras[0:len(axes)],TSX_dates,labels,label_x,label_y):
    image = load_imag(img)
    image_crop = image[5000:10000,1000:4000]
    a.imshow(np.fliplr(np.rot90(image_crop,3)),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.annotate(date.strftime('%Y-%m-%d'),xy=[lab_x+0.0,lab_y-0.795],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.set_axis_off()
ax33.set_axis_off()
fig.subplots_adjust(wspace=0.01,hspace=0.01)



In [6]:
# normalise to 0 - 1
def normalise_arr(arr):
    """Normalises N-D numpy array to values between 0 and 1 where the minimum value 
       in the original array becomes 0 and the maximum value becomes 1.

    Args:
        arr (Numpy array): ND numeric array
    """
    return (arr-np.min(arr))/(np.max(arr)-np.min(arr))

# compare images
def compare_RMLI(file1, file2):
    image1 = load_imag(file1)
    image2 = load_imag(file2)
    # caluclate difference
    image_diff = image2 - image1

    # normalise
    RMLI1_arr_norm = normalise_arr(image1)
    RMLI2_arr_norm = normalise_arr(image2)
    RMLI_diff_arr_norm = normalise_arr(image_diff)

    # create rgb composite
    rgb_image = np.dstack([RMLI1_arr_norm,
                       RMLI2_arr_norm,
                       RMLI_diff_arr_norm])
    
    return image1, image2, rgb_image

In [9]:
%matplotlib osx
import string
# display image
plt.close('all')

fig = plt.figure(figsize=(8,8))

gs = GridSpec(4, 4, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # 
ax01 = fig.add_subplot(gs[0, 1]) # 
ax02 = fig.add_subplot(gs[0, 2]) # 
ax03 = fig.add_subplot(gs[0, 3]) # 


ax10 = fig.add_subplot(gs[1, 0]) # 
ax11 = fig.add_subplot(gs[1, 1]) # 
ax12 = fig.add_subplot(gs[1, 2]) # 
ax13 = fig.add_subplot(gs[1, 3]) # 

ax20 = fig.add_subplot(gs[2, 0]) # 
ax21 = fig.add_subplot(gs[2, 1]) # 
ax22 = fig.add_subplot(gs[2, 2]) # 
ax23 = fig.add_subplot(gs[2, 3]) # 

ax30 = fig.add_subplot(gs[3, 0]) # 
ax31 = fig.add_subplot(gs[3, 1]) # 
ax32 = fig.add_subplot(gs[3, 2]) # 
ax33 = fig.add_subplot(gs[3, 3]) # 


axes = [ax00,ax01,ax02,ax03,ax10,ax11,ax12,ax13,ax20,ax21,ax22,ax23,ax30,ax31,ax32,ax33]
labels = string.ascii_uppercase[0:len(axes)]
label_x = [0.02,0.02,0.02,0.02,
           0.02,0.02,0.02,0.02,
           0.02,0.02,0.02,0.02,
           0.02,0.02,0.02,0.02]
label_y = [0.87,0.87,0.87,0.87,
           0.87,0.87,0.87,0.87,
           0.87,0.87,0.87,0.87,
           0.87,0.87,0.87,0.87]

bbox = dict(fc="1.0",alpha=0.5)
for i,(a,img,label,lab_x,lab_y) in enumerate(zip(axes,TSX_RMLI_ras[0:len(axes)],labels,label_x,label_y)):
    print(i)
    image = load_imag(img)
    image1, image2, image_rgb = compare_RMLI(TSX_RMLI_ras[0], img)
    image_crop = image_rgb[5000:10000,1000:3000,:]
    a.imshow(np.rot90(image_crop,3))
    # a.imshow(np.rot90(image_crop,3),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.annotate
    a.set_axis_off()
ax33.set_axis_off()


fig.subplots_adjust(hspace=0.01,wspace=0.01)

0


/var/folders/7b/9xvft5dd0455jpkm_r7rgr3w0000gp/T/ipykernel_8232/3936169127.py:9: RuntimeWarning: invalid value encountered in divide
  return (arr-np.min(arr))/(np.max(arr)-np.min(arr))


1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [6]:
# plot TSX image 0, 4, 12 and 13 in a row

%matplotlib osx
import string
# display image
plt.close('all')

fig = plt.figure(figsize=(8,8))

gs = GridSpec(1, 4, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # 
ax01 = fig.add_subplot(gs[0, 1]) # 
ax02 = fig.add_subplot(gs[0, 2]) # 
ax03 = fig.add_subplot(gs[0, 3]) # 




axes = [ax00,ax01,ax02,ax03]
labels = string.ascii_uppercase[0:len(axes)]
label_x = [0.02,0.02,0.02,0.02,]
label_y = [0.87,0.87,0.87,0.87,]

TSX_sel = [TSX_RMLI_ras[0],TSX_RMLI_ras[4],TSX_RMLI_ras[12],TSX_RMLI_ras[13],]
TSX_dates_sel = [TSX_dates[0],TSX_dates[4],TSX_dates[12],TSX_dates[13],]
bbox = dict(fc="1.0",alpha=0.5)
for (a,img,date, label,lab_x,lab_y) in zip(axes,TSX_sel,TSX_dates_sel,labels,label_x,label_y):
    image = load_imag(img)
    image_crop = image[5000:10000,1000:4000]
    a.imshow(np.fliplr(np.rot90(image_crop,3)),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.annotate(date.strftime('%Y-%m-%d'),xy=[lab_x+0.0,lab_y-0.795],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.set_axis_off()
fig.subplots_adjust(wspace=0.01,hspace=0.01)


In [29]:
# # display image
# %matplotlib osx
# plt.close('all')

# fig = plt.figure(figsize=(8,8))

# gs = GridSpec(2, 3, figure=fig,height_ratios=[10,20])

# ax00 = fig.add_subplot(gs[0, 0]) # image1
# ax01 = fig.add_subplot(gs[0, 1]) # image2
# ax02 = fig.add_subplot(gs[0, 2]) # composite

# ax10 = fig.add_subplot(gs[1, 0]) # image1
# ax11 = fig.add_subplot(gs[1, 1]) # image2
# ax12 = fig.add_subplot(gs[1, 2]) # composite

# bbox = dict(fc="1.0",alpha=0.5)
# cmaps = ['Greys_r', 'Greys_r','None','Greys_r', 'Greys_r','None']
# labels = ['A','B','C','D','E','F']
# label_x = [0.02,0.02,0.02,0.02,0.02,0.02]
# label_y = [0.91,0.91,0.91,0.955,0.955,0.955]
# for i, (a,image,cmap,label,lab_x,lab_y) in enumerate(zip([ax00,ax01,ax02,ax10,ax11,ax12],[CSK_rmli1_crop, CSK_rmli2_crop, CSK_rgb_crop,TSX_rmli1_crop, TSX_rmli2_crop, TSX_rgb_crop],cmaps,labels,label_x,label_y)):
#     print(i)
#     if cmap == 'None':
#         a.imshow(image)
#     else:
#         a.imshow(image,cmap=cmap)
    
#     a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
#     a.annotate
#     a.set_axis_off()

# fig.tight_layout()

0
1
2
3
4
5
